#### Angie

In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [2]:
!pip install selenium
!pip install webdriver_manager


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# this library is to manipulate browser
from selenium import webdriver

# it allows you to work with differen versions of drivers
# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import re
import time 
import pandas as pd
from selenium.webdriver.common.by import By

In [3]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.execute_script("document.body.style.zoom='100%'")

url = 'https://infogob.jne.gob.pe/Eleccion'
driver.get(url)

In [4]:
#All the elements for the loop and some examples. 
selection = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
selection.click()

In [5]:
presidential_elections = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
presidential_elections.click()

In [6]:
selection_2 = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
selection_2.click()

In [7]:
elections_2021 = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[2]')
elections_2021.click()

In [8]:
see_data = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button/span')
see_data.click()

In [9]:
candidates_results = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
candidates_results.click()

#### Marilia

In [10]:
# Steps the loop will follow:
# 1. Initialize the web scraper using Selenium and configure the Chrome browser.
# 2. Iterate through a series of elections (numbers 2 to 26).
# 3. For each election, navigate to the corresponding page, extract the name, and access the results.
# 4. Process the HTML results table, clean it, and select the relevant columns as required by the task.
# 5. Append the cleaned data to a list, and finally, concatenate them into a DataFrame saved in an Excel file.
# 6. Close the browser upon completing the scraping process.


In [11]:
# Store all_tables
all_tables = {}


In [12]:
#Bucle step by step:

# Define a function to fetch election results table
def get_election_table(driver, number):
    # Open the election results webpage
    driver.get(url)
    
    # Select the first option select
    selection_1 = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
    selection_1.click()
    time.sleep(1)
    
    # Select the "Presidential Elections" option
    presidential_elections = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
    presidential_elections.click()
    time.sleep(1)

    # Select the second option select
    selection_2 = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
    selection_2.click()
    time.sleep(1)

    # Select the specific election option
    election_options = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{}]'.format(number))
    election_options.click()
    election_name = election_options.text
    time.sleep(1)

    # Click the "See Data" button
    see_data = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
    see_data.click()
    time.sleep(2)

    # Go to the "Candidates Results" section
    candidates_results = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
    candidates_results.click()
    time.sleep(1)

    # Extract the HTML content of the election results table
    table_ID_cleaned = driver.find_element(By.ID,'gridEleccionResultadosElectorales')
    table_html = table_ID_cleaned.get_attribute('innerHTML')

    # Clean the HTML content to make it compatible with pandas
    table_head = "<table id=\"gridEleccionResultadosElectorales\" class=\"table table-hover table-striped\">"
    cleaned_table = table_head + table_html[21:-1]

    # Read the HTML table into a DataFrame
    table = pd.read_html(cleaned_table)
    
    # Juan Diego Linares
    # Lastly, as it is stated in the task, we have to create a new colum called "elecciones" and we also have to get only the colums "Organización Política" and "Total Votos"
    # With this, we will get the requested table.
    cleaned_table = table[0].iloc[:, : ]
    cleaned_table = cleaned_table.loc[:, ['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]
    cleaned_table.insert(0, 'Elecciones', election_name)

    return cleaned_table

In [14]:
# Juan Diego Linares
#Preparing the bucle using the function previously defined.

# First we configure the driver and the URL where we want to get the tables from.
driver = webdriver.Chrome()
driver.maximize_window()
url = 'https://infogob.jne.gob.pe/Eleccion'

# Now we create a list where we want to store individual DataFrames

dataframes_elections = []

# Giordano Medina Montes

# We loop through to all the multiple elections ( we have 26 options in total inside of the section "Elección"). This will iterate trough all of them with the objective of recolecting all the tables.
for number in range(2, 27):
    election_table = get_election_table(driver, number)
    dataframes_elections.append(election_table)

# Finally, we join all the tables into one dataframe. Next, we will present the joined table and the route to export  it as an excel file.

# We concatenate all the DataFrames into a single one as we was stated.
final_dataframe = pd.concat(dataframes_elections, ignore_index=True)

# Now we can save the final DataFrame as an Excel file and as an example, we will use the next direction, but it can be changed.

save_path = r'group_5_ass_5_2024.xlsx'
final_dataframe.to_excel(save_path, index=False)

# Lastly, we close the driver when the webscrapping process is finished and we will print the dataframe to see what we have got.
# As we can see, we got what was asked and now we
driver.quit()

final_dataframe



,Elecciones,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,PRESIDENCIAL 2021 - 2DA VUELTA,PARTIDO POLITICO NACIONAL PERU LIBRE,8836380
1,PRESIDENCIAL 2021 - 2DA VUELTA,FUERZA POPULAR,8792117
2,PRESIDENCIAL 2021 - 2DA VUELTA,VOTOS EN BLANCO,121489
3,PRESIDENCIAL 2021 - 2DA VUELTA,VOTOS NULOS,1106816
4,PRESIDENCIAL 2021,PARTIDO POLITICO NACIONAL PERU LIBRE,2724752
...,...,...,...
152,PRESIDENCIAL 1936,PARTIDO REPUBLICANO,30803
153,PRESIDENCIAL 1931,UNION REVOLUCIONARIA,152149
154,PRESIDENCIAL 1931,PARTIDO APRISTA PERUANO,106088
155,PRESIDENCIAL 1931,PARTIDO DESCENTRALISTA,21950
